## Installing necessary libraries

In [50]:
%%bash

# Dependencies
pip install docx2txt
pip install strsimpy
pip install python-docx
pip install pandas

if ls docx2csv >/dev/null 2>&1; then
    echo "docx2csv exists."
else
    echo "Folder does not exist. Cloning docx2csv."
    git clone https://github.com/ivbeg/docx2csv.git
fi

Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
docx2csv exists.


In [51]:
%%bash

source .env
cd docx2csv && echo "$PASSWORD" | sudo -S python3 setup.py install

running install


[sudo] password for arragon: /usr/lib/python3/dist-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/usr/lib/python3/dist-packages/setuptools/command/easy_install.py:158: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


running bdist_egg
running egg_info
writing docx2csv.egg-info/PKG-INFO
writing dependency_links to docx2csv.egg-info/dependency_links.txt
writing entry points to docx2csv.egg-info/entry_points.txt
writing requirements to docx2csv.egg-info/requires.txt
writing top-level names to docx2csv.egg-info/top_level.txt
reading manifest file 'docx2csv.egg-info/SOURCES.txt'
adding license file 'LICENSE'
adding license file 'AUTHORS.rst'
writing manifest file 'docx2csv.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/docx2csv
copying build/lib/docx2csv/converter.py -> build/bdist.linux-x86_64/egg/docx2csv
copying build/lib/docx2csv/core.py -> build/bdist.linux-x86_64/egg/docx2csv
copying build/lib/docx2csv/__init__.py -> build/bdist.linux-x86_64/egg/docx2csv
copying build/lib/docx2csv/__main__.py -> build/bdist.linux-x86_64/egg/docx2csv
byte-compiling build/bd

In [52]:
# ----- TEST DATA INPUT -----

# Computer Science Test Data.
# CURRENT_MAPPING="Lists_ComputerScience.docx"
# ORIGINAL_MAPPING="Original-Mapping-ComputerScience.csv"

# InformationSecurity Test Data.
CURRENT_MAPPING="Lists_MonashEngineering.docx"
ORIGINAL_MAPPING="Original-Mapping-MonashEngineering.csv"

In [53]:
# importing necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [54]:
# extract tables from word document
from docx2csv import extract_tables, extract
tables = extract_tables(CURRENT_MAPPING)

In [55]:
from docx import Document
document = Document(CURRENT_MAPPING)

In [56]:
def read_docx_table(document,table_num):
  table = document.tables[table_num-1]
  data = [[cell.text for cell in row.cells] for row in table.rows]
  df = pd.DataFrame(data)
  return df

## PLO TABLE

In [57]:
# Creating a dataframe for PLOs and it will accept 'n' number of PLOs
table_num=1
df = read_docx_table(document,table_num)
df.head(n=12)

,0,1
0,PO1,"Apply knowledge of mathematics, natural scienc..."
1,PO2,"Identify, formulate, survey research literatur..."
2,PO3,Design solutions for complex Software engineer...
3,PO4,Conduct investigations of complex Software eng...
4,PO5,"Create, select and apply appropriate technique..."
5,PO6,Apply reasoning informed by contextual knowled...
6,PO7,Understand and evaluate the sustainability and...
7,PO8,Apply ethical principles and commit to profess...
8,PO9,Communicate effectively on complex Software en...
9,PO10,"Function effectively as an individual, and as ..."


In [58]:
q1 = df.copy()
df_po = df.copy()

In [59]:
# assigning count vectorizer
count_vectorizer = CountVectorizer(stop_words='english', min_df=0.005)

In [60]:
# Remove integers

# Data preprocessing for PLO dataframe
q1[1] = q1[1].str.lower()
corpus = q1[1].tolist()
corpii = count_vectorizer.fit_transform(corpus)
corpus

['apply knowledge of mathematics, natural science, engineering fundamentals and specialization in software engineering to the solution of complex engineering problems;\n',
 'identify, formulate, survey research literature and analyze complex software engineering problems reaching substantiated conclusions using first principles of mathematics, natural sciences and engineering sciences;\n',
 'design solutions for complex software engineering problems and design systems, components or processes that meet specified needs;',
 'conduct investigations of complex software engineering problems using research-based knowledge and research methods including design of experiments, analysis and interpretation of data, and synthesis of information to provide valid conclusions;',
 'create, select and apply appropriate techniques, resources, and modern engineering and it tools, including prediction and modelling, to complex software engineering problems, with an understanding of the limitations;',
 'a

In [61]:
corpii

<12x128 sparse matrix of type '<class 'numpy.int64'>'
	with 182 stored elements in Compressed Sparse Row format>

In [62]:
# extracting features names from PLO table
feature_names = count_vectorizer.get_feature_names_out()
feature_names

array(['ability', 'able', 'activities', 'analysis', 'analyze', 'apply',
       'appropriate', 'assess', 'based', 'broadest', 'change', 'clear',
       'commit', 'communicate', 'community', 'complex', 'components',
       'comprehend', 'conclusions', 'conduct', 'consequent', 'context',
       'contexts', 'contextual', 'create', 'cultural', 'data', 'decision',
       'demonstrate', 'design', 'disciplinary', 'diverse',
       'documentation', 'economic', 'effective', 'effectively', 'engage',
       'engineering', 'environmental', 'ethical', 'ethics', 'evaluate',
       'experiments', 'formulate', 'function', 'fundamentals', 'health',
       'identify', 'impact', 'including', 'independent', 'individual',
       'information', 'informed', 'instructions', 'interpretation',
       'investigations', 'issues', 'knowledge', 'large', 'leader',
       'learning', 'legal', 'life', 'limitations', 'literature', 'long',
       'make', 'making', 'manage', 'management', 'mathematics', 'meet',
       'me

In [63]:
len(feature_names)

128

In [64]:
# Converting features to vector form and create a dataframe
X1 = pd.DataFrame(corpii.toarray(), columns=feature_names)

## CLO TABLE

In [65]:
# Creating a dataframe for CLOs and it will accept 'n' number of CLOs
table_num=2
df1 = read_docx_table(document,table_num)
p1 = df1.copy()
df_clo = df1.copy()

In [66]:
# Data preprocessing for CLO dataframe
p1[1] = p1[1].str.lower()
corpus11 = p1[1].tolist()
corpii11 = count_vectorizer.fit_transform(corpus11)

In [67]:
# extracting features names from CLO table
feature_names1 = count_vectorizer.get_feature_names_out()

In [68]:
len(feature_names1)

454

In [69]:
# Converting features to vector form and create a dataframe
X2 = pd.DataFrame(corpii11.toarray(), columns=feature_names1)
X2

,ability,abstract,acceptable,accordance,achieve,achieved,actions,activities,adopt,advanced,...,visualisations,vocabulary,wired,wireless,work,working,worst,write,writing,written
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
83,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
84,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
85,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
X2.head()

,ability,abstract,acceptable,accordance,achieve,achieved,actions,activities,adopt,advanced,...,visualisations,vocabulary,wired,wireless,work,working,worst,write,writing,written
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
# adding column index to the CLO table
U2 = pd.concat([df1[0], X2], axis=1)
U2.set_index(0, inplace=True)

In [72]:
U2.head()

,ability,abstract,acceptable,accordance,achieve,achieved,actions,activities,adopt,advanced,...,visualisations,vocabulary,wired,wireless,work,working,worst,write,writing,written
0,,,,,,,,,,,,,,,,,,,,,
LO1-MA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LO2-MA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LO3-MA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LO4-MA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LO5-MA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
# adding column index to the PLO table
U1 = pd.concat([df[0], X1], axis=1)
U1.set_index(0, inplace=True)

In [74]:
U1

,ability,able,activities,analysis,analyze,apply,appropriate,assess,based,broadest,...,teams,techniques,technological,tools,understand,understanding,using,valid,work,write
0,,,,,,,,,,,,,,,,,,,,,
PO1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PO2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
PO3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PO4,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,1,1,0,0
PO5,0,0,0,0,0,1,1,0,0,0,...,0,1,0,1,0,1,0,0,0,0
PO6,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
PO7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
PO8,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PO9,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## Intersection method for both CLOs and PLOs

### Generalised list of words

In [75]:
append_words = list(map(str.lower,['Cite', 'Define', 'Describe', 'Draw', 'Enumerate', 'Identify' 'Index', 'Indicate', 'Label', 'List', 'Match', 'Meet', 'Name', 'Outline', 'Point', 'Quote', 'Read', 'Recall', 'Recite', 'Recognize', 'Record', 'Repeat', 'Reproduce','Review',
'Select', 'State', 'Study', 'Tabulate', 'Trace', 'Write', 'Add', 'Approximate', 'Articulate', 'Associate', 'Characterize', 'Clarify', 'Classify', 'Compare', 'Compute', 'Contrast', 'Convert', 'Defend', 'Detail', 'Differentiate',
'Discuss', 'Distinguish', 'Elaborate', 'Estimate', 'Example', 'Explain', 'Express', 'Extend', 'Extrapolate', 'Factor', 'Generalize', 'Give', 'Infer', 'Interact', 'Interpolate', 'Interpret', 'Observe', 'Paraphrase', 'Picture graphically',
'Predict', 'Rewrite', 'Subtract', 'Summarize', 'Translate', 'Visualize', 'Acquire', 'Adapt', 'Allocate', 'Alphabetize', 'Apply', 'Ascertain', 'Assign', 'Attain', 'Avoid', 'Back up', 'Calculate', 'Capture', 'Change', 'Complete', 'Construct',
'Customize', 'Demonstrate', 'Depreciate', 'Derive', 'Determine', 'Diminish', 'Discover', 'Employ', 'Examine', 'Exercise', 'Explore', 'Expose', 'Figure', 'Graph', 'Handle', 'Illustrate', 'Interconvert', 'Investigate', 'Manipulate', 'Modify',
'Operate', 'Personalize', 'Plot','Practice', 'Prepare', 'Price', 'Process', 'Produce', 'Project', 'Provide', 'Relate', 'Round off', 'Sequence', 'Show', 'Simulate', 'Sketch', 'Solve', 'Subscribe', 'Transcribe', 'Use', 'Analyze', 'Audit',
'Blueprint', 'Breadboard', 'Break down', 'Confirm', 'Correlate', 'Detect', 'Diagnose', 'Diagram', 'Discriminate', 'Dissect', 'Document', 'Ensure', 'Figure out', 'File', 'Group', 'Interrupt', 'Inventory', 'Layout', 'Manage', 'Maximize',
'Minimize', 'Optimize', 'Order', 'Point out', 'Prioritize', 'Proofread', 'Query', 'Separate', 'Subdivide', 'Train', 'Transform', 'Appraise', 'Assess', 'Conclude', 'Counsel', 'Criticize', 'Critique', 'Evaluate', 'Grade', 'Hire', 'Judge',
'Justify', 'Measure', 'Prescribe', 'Rank', 'Rate', 'Recommend', 'Release', 'Support', 'Test', 'Validate', 'Verify', 'Abstract', 'Animate', 'Arrange', 'Assemble', 'Budget', 'Categorize', 'Code', 'Combine', 'Compile', 'Compose', 'Cope',
'Correspond', 'Create', 'Cultivate', 'Debug', 'Depict', 'Design', 'Develop', 'Devise', 'Dictate', 'Enhance', 'Facilitate', 'Format', 'Formulate', 'Generate', 'Import', 'Improve', 'Incorporate', 'Integrate', 'Interface', 'Join', 'Lecture',
'Model', 'Network', 'Organize', 'Overhaul', 'Plan', 'Portray', 'Program', 'Rearrange', 'Reconstruct', 'Reorganize', 'Revise', 'Specify']))

In [76]:
# using + operator to concat the generalised list of words to the PLO list
train_column = list(feature_names) + append_words

In [77]:
# CLO list of words
test_column = feature_names1
test_column

array(['ability', 'abstract', 'acceptable', 'accordance', 'achieve',
       'achieved', 'actions', 'activities', 'adopt', 'advanced', 'agreed',
       'algebra', 'algorithmic', 'algorithms', 'allocate', 'analyse',
       'analysis', 'applications', 'apply', 'approach', 'approaches',
       'appropriate', 'appropriately', 'architectural', 'architectures',
       'artifacts', 'aspects', 'assembly', 'assess', 'assumptions',
       'assurance', 'attributes', 'australia', 'automated',
       'availability', 'available', 'backend', 'balances', 'based',
       'basic', 'basis', 'beam', 'behaviour', 'behaviours', 'best', 'big',
       'board', 'boundaries', 'brief', 'business', 'c3', 'c4', 'c5', 'c6',
       'calculus', 'capability', 'carry', 'case', 'chemical', 'choose',
       'circuit', 'circuits', 'civil', 'classes', 'client', 'clients',
       'code', 'codes', 'collaboration', 'collection', 'combination',
       'commit', 'commitment', 'common', 'communicate', 'communicated',
       'comm

In [78]:
# Intersection method for extracting common column names from the tables (both CLO AND PLO)
# comparing whether the CLO column name is present in the PLO column names or not
train_column = list(feature_names) + append_words # (PLO table ) (# using + operator to concat PLO words and list of generalized words)
test_column = list(feature_names1)   # (CLO table)

In [79]:
# This is the column names from both the tables (using intersection)
common_column = list(set(train_column).intersection(set(test_column)))
common_column

['context',
 'evaluate',
 'contrast',
 'project',
 'activities',
 'determine',
 'understand',
 'ensure',
 'plan',
 'produce',
 'assess',
 'leader',
 'impact',
 'sciences',
 'define',
 'support',
 'management',
 'apply',
 'network',
 'understanding',
 'processes',
 'norms',
 'meet',
 'document',
 'resources',
 'process',
 'including',
 'solution',
 'ethics',
 'tools',
 'review',
 'communicate',
 'explain',
 'mathematics',
 'abstract',
 'systems',
 'debug',
 'documentation',
 'sustainability',
 'data',
 'relevant',
 'consequent',
 'based',
 'compare',
 'manage',
 'identify',
 'complex',
 'member',
 'analysis',
 'principles',
 'life',
 'appropriate',
 'write',
 'fundamentals',
 'needs',
 'demonstrate',
 'state',
 'knowledge',
 'commit',
 'select',
 'organize',
 'investigations',
 'allocate',
 'issues',
 'cultural',
 'components',
 'health',
 'limitations',
 'solve',
 'devise',
 'justify',
 'practice',
 'use',
 'discuss',
 'investigate',
 'solutions',
 'safety',
 'software',
 'problems',
 

In [80]:
print(common_column)

['context', 'evaluate', 'contrast', 'project', 'activities', 'determine', 'understand', 'ensure', 'plan', 'produce', 'assess', 'leader', 'impact', 'sciences', 'define', 'support', 'management', 'apply', 'network', 'understanding', 'processes', 'norms', 'meet', 'document', 'resources', 'process', 'including', 'solution', 'ethics', 'tools', 'review', 'communicate', 'explain', 'mathematics', 'abstract', 'systems', 'debug', 'documentation', 'sustainability', 'data', 'relevant', 'consequent', 'based', 'compare', 'manage', 'identify', 'complex', 'member', 'analysis', 'principles', 'life', 'appropriate', 'write', 'fundamentals', 'needs', 'demonstrate', 'state', 'knowledge', 'commit', 'select', 'organize', 'investigations', 'allocate', 'issues', 'cultural', 'components', 'health', 'limitations', 'solve', 'devise', 'justify', 'practice', 'use', 'discuss', 'investigate', 'solutions', 'safety', 'software', 'problems', 'need', 'community', 'environmental', 'multi', 'work', 'engineering', 'techniqu

In [81]:
len(common_column)

105

In [82]:
# Filter the common column values from the CLO table
U3 = U2.filter(list(common_column), axis=1)

In [83]:
U3.head()

,context,evaluate,contrast,project,activities,determine,understand,ensure,plan,produce,...,code,model,develop,ability,verify,diverse,legal,responsibilities,natural,ethical
0,,,,,,,,,,,,,,,,,,,,,
LO1-MA,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
LO2-MA,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
LO3-MA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LO4-MA,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
LO5-MA,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [84]:
# extracting first row from PLO table and make a dataframe
Cs = []
for x in range(len(df)):
    Cs.append(U1.loc[['PO'+str(x+1)]])

In [85]:
# Concatenating these extracted each PLOs with 'n' number of CLOs
Dds = []
for x in range(len(df)):
    Dds.append(pd.concat([Cs[x],U3], sort=True))

In [86]:
# Filling the nan values of the concatenated dataframes
Ds = []
for x in range(len(df)):
    Ds.append(Dds[x].fillna(0))

## Calculate Cosine similarity

In [87]:
# Calculate cosine similarity for concatenated dataframes and create a new dataframe
for x in range(len(df)):
    Dds[x] = pd.DataFrame(cosine_similarity(Ds[x], dense_output=True))
Ds

[        ability  able  abstract  activities  allocate  analysis  analyze  \
 0                                                                          
 PO1           0   0.0       0.0           0       0.0         0      0.0   
 LO1-MA        0   0.0       0.0           0       0.0         0      0.0   
 LO2-MA        0   0.0       0.0           0       0.0         0      0.0   
 LO3-MA        0   0.0       0.0           0       0.0         0      0.0   
 LO4-MA        0   0.0       0.0           0       0.0         0      0.0   
 ...         ...   ...       ...         ...       ...       ...      ...   
 LO2-MA        0   0.0       0.0           0       0.0         0      0.0   
 LO3-MA        0   0.0       0.0           0       0.0         0      0.0   
 LO4-MA        0   0.0       0.0           0       0.0         0      0.0   
 LO5-MA        0   0.0       0.0           0       0.0         0      0.0   
 LO6-MA        0   0.0       0.0           1       0.0         0      0.0   

In [88]:
# Extract the '0'th column because it has the CLO-PLO  cosine similarity values. We are neglecting the remaining ones.
# Renaming the '0'th column name to 'Pn' ['P1, P2, P3, P4, ... 'Pn']
for x in range(len(df)):
    Dds[x].rename(columns = {0 :'PO'+str(x+1)}, inplace = True)

Dds

[         PO1         1         2         3         4    5    6         7  \
 0   1.000000  0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.372678   
 1   0.000000  1.000000  0.577350  0.000000  0.707107  0.0  0.0  0.000000   
 2   0.000000  0.577350  1.000000  0.666667  0.408248  0.0  0.0  0.000000   
 3   0.000000  0.000000  0.666667  1.000000  0.000000  0.0  0.0  0.000000   
 4   0.000000  0.707107  0.408248  0.000000  1.000000  0.0  0.0  0.000000   
 ..       ...       ...       ...       ...       ...  ...  ...       ...   
 83  0.070711  0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.000000   
 84  0.212132  0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.105409   
 85  0.461880  0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.258199   
 86  0.269680  0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.100504   
 87  0.521749  0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.222222   
 
            8         9  ...        78        79        80        81      

In [89]:
# Concatenating each  '0'th column from different cosine similarity dataframes
Ddn = []
for x in range(len(df)):
    Ddn.append(Dds[x]['PO'+str(x+1)])

d = pd.concat(Ddn, axis=1)
d

,PO1,PO2,PO3,PO4,PO5,PO6,PO7,PO8,PO9,PO10,PO11,PO12
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.298142,0.230940,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.447214,0.230940,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
83,0.070711,0.064550,0.081650,0.000000,0.212132,0.063246,0.079057,0.100000,0.000000,0.000000,0.000000,0.182574
84,0.212132,0.129099,0.081650,0.063246,0.141421,0.569210,0.316228,0.400000,0.121716,0.000000,0.000000,0.182574
85,0.461880,0.263523,0.200000,0.154919,0.288675,0.309839,0.839146,0.244949,0.198762,0.000000,0.143223,0.149071
86,0.269680,0.184637,0.077850,0.060302,0.202260,0.361814,0.226134,0.953463,0.116052,0.000000,0.000000,0.261116


In [90]:
# '0'th column gives us 1 which means each PLO map with own PLO.
# So we are removing that column.
dd = d[1:]
dd

,PO1,PO2,PO3,PO4,PO5,PO6,PO7,PO8,PO9,PO10,PO11,PO12
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.298142,0.230940,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.447214,0.230940,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
83,0.070711,0.064550,0.081650,0.000000,0.212132,0.063246,0.079057,0.100000,0.000000,0.000000,0.000000,0.182574
84,0.212132,0.129099,0.081650,0.063246,0.141421,0.569210,0.316228,0.400000,0.121716,0.000000,0.000000,0.182574
85,0.461880,0.263523,0.200000,0.154919,0.288675,0.309839,0.839146,0.244949,0.198762,0.000000,0.143223,0.149071
86,0.269680,0.184637,0.077850,0.060302,0.202260,0.361814,0.226134,0.953463,0.116052,0.000000,0.000000,0.261116


In [91]:
# resetting index
dd.reset_index(inplace = True)
dd.drop(['index'], axis=1, inplace = True)
dd

/tmp/ipykernel_11207/1086022003.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dd.drop(['index'], axis=1, inplace = True)


,PO1,PO2,PO3,PO4,PO5,PO6,PO7,PO8,PO9,PO10,PO11,PO12
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.298142,0.230940,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.447214,0.230940,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
82,0.070711,0.064550,0.081650,0.000000,0.212132,0.063246,0.079057,0.100000,0.000000,0.000000,0.000000,0.182574
83,0.212132,0.129099,0.081650,0.063246,0.141421,0.569210,0.316228,0.400000,0.121716,0.000000,0.000000,0.182574
84,0.461880,0.263523,0.200000,0.154919,0.288675,0.309839,0.839146,0.244949,0.198762,0.000000,0.143223,0.149071
85,0.269680,0.184637,0.077850,0.060302,0.202260,0.361814,0.226134,0.953463,0.116052,0.000000,0.000000,0.261116


In [92]:
# print the matrix
print(dd)

         PO1       PO2       PO3       PO4       PO5       PO6       PO7  \
0   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1   0.000000  0.000000  0.298142  0.230940  0.000000  0.000000  0.000000   
2   0.000000  0.000000  0.447214  0.230940  0.000000  0.000000  0.000000   
3   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
..       ...       ...       ...       ...       ...       ...       ...   
82  0.070711  0.064550  0.081650  0.000000  0.212132  0.063246  0.079057   
83  0.212132  0.129099  0.081650  0.063246  0.141421  0.569210  0.316228   
84  0.461880  0.263523  0.200000  0.154919  0.288675  0.309839  0.839146   
85  0.269680  0.184637  0.077850  0.060302  0.202260  0.361814  0.226134   
86  0.521749  0.340207  0.258199  0.200000  0.372678  0.333333  0.416667   

         PO8       PO9      PO10      PO11      PO12  
0   0.000000  0.000000  0.000000

In [93]:
## This code loads the vector file into the word_vectors variable
## Download the vector file from https://fasttext.cc/docs/en/english-vectors.html (first file on the website), unzip the file and store in your local development folder
## Note: This piece of code may take upto an hour or two to run depending on your pc specs.
## My i5 8th gen with 8gig ram took 58mins to run.

# from gensim.models import KeyedVectors

# # Path to the downloaded .vec file
# path_to_vectors = 'wiki-news-300d-1M.vec'
# # path_to_vectors = 'wiki.en.vec'
# # Load the word vectors
# word_vectors = KeyedVectors.load_word2vec_format(path_to_vectors)

# # Find similar words
# similar_words = word_vectors.most_similar('cat')

# # Calculate word similarity
# similarity = word_vectors.similarity('cat', 'dog')

# # Perform vector arithmetic
# result = word_vectors['king'] - word_vectors['man'] + word_vectors['woman']


### LO Mapper Setup:

In [94]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from gensim.test.utils import common_texts

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import pandas as pd
import csv
import math

# import pprint
import re
import json

import spacy

# Initialises dictionary containing American to UK spelling translations
american_to_british_dict = {}
american_to_british_path = "American-British-English-Translator.json"
with open(american_to_british_path, "r") as file:
    data = file.read()
american_to_british_dict = json.loads(data)


def britishise(sentence):
    """
    Convert words in a sentence to UK spelling to ensure consistency

    Input:
        sentence: An array of strings

    Output:
        sentence: An array of strings, which have been converted to UK spelling
    """

    for j in range(len(sentence)):
        try:
            sentence[j] = american_to_british_dict[sentence[j].lower()]
        except:
            pass

    return sentence

lemmatizer = WordNetLemmatizer()


def verb_classifier(verbs_file_path):  #
    """
    Takes an excel spreadsheet containing verbs, classifies the verbs and stores it into a Dataframe.


    Inputs:
        verbs_file_path: A string that contains the path to the excel spreadsheet to be read

    Outputs:
        domain_levels: A DataFrame which contains all the verbs from the spreadsheet classified into their respective levels
    """

    xlsx = pd.ExcelFile(verbs_file_path, engine="openpyxl")

    sheet_names = xlsx.sheet_names  # Get a list of sheet names

    # Create an empty dictionary to store DataFrames for each sheet
    dfs = {}

    for sheet_name in sheet_names:
        df = xlsx.parse(sheet_name)  # For XLSX files

        # Store the DataFrame in the dictionary
        dfs[sheet_name] = df

    domain_levels = pd.concat(dfs)
    duplicate_checklist = []

    # Iterate over all values in the spreadsheet
    for i in range(domain_levels.shape[0]):
        for j in range(domain_levels.shape[1]):
            cell_value = domain_levels.iloc[i, j]
            if not pd.isna(
                cell_value
            ):  # Format verbs (lower case, UK spelling, lemmatised format)
                cell_value_lower = cell_value.lower()
                verb_brit = britishise([cell_value_lower])[0]
                verb = lemmatizer.lemmatize(verb_brit, pos="v")

                if (
                    verb not in duplicate_checklist
                ):  # Check if the verb is already mapped
                    domain_levels.iloc[i, j] = verb
                    duplicate_checklist.append(verb)
                else:
                    domain_levels.iloc[i, j] = float("nan")

    domain_levels = domain_levels.dropna(how="all")
    return domain_levels

# Convert mapped_verbs DataFrames which is used to find verbs at a certain level, to a 'dictionary' to lookup the level of a verb
def generate_verb_list(mapped_verbs):
    tp_arr = []
    levels = []
    for t_key, t_item in mapped_verbs.items():
        print(t_item)
        columns = t_item.columns.values
        for x in range(t_item.shape[0]):
            for y in range(t_item.shape[1]):
                verb = t_item.iloc[x, y]
                if not pd.isna(verb):
                    tp_arr.append((t_key, verb))
                    levels.append(columns[y])

    index = pd.MultiIndex.from_tuples(tp_arr)
    verb_list = pd.DataFrame(levels, index=index, columns=["Level"])
    return verb_list

# Load the English language model in spaCy
# nlp = spacy.load('en_core_web_sm')
nlp = spacy.load("en_core_web_trf", exclude=["ner"])


## Function to identify verbs in a sentence
def identify_verbs(sentence):
    """
    Identify verbs within a sentence and lemmatise them (convert them into their base word)

    Inputs:
        sentence: A string

    Outputs:
        verbs: An array of strings representing identified verbs in their lemmatised form
    """

    # Process the sentence using spaCy
    doc = nlp(sentence)

    # Extract the verbs from the processed sentence
    verbs = [token.lemma_ for token in doc if token.pos_ == "VERB"]

    return verbs


def extract_columns(file_path, columns):
    extracted_data = {}

    with open(file_path, "r") as file:
        reader = csv.reader(file)
        headers = next(reader)  # Read the headers

        # Check if all specified columns exist in the CSV file
        for column in columns:
            if column not in headers:
                raise ValueError(f"Column '{column}' not found in the CSV file.")

        # Initialize separate arrays for each column
        for column in columns:
            extracted_data[column] = []

        # Extract data from specified columns
        for row in reader:
            for column in columns:
                column_index = headers.index(column)
                extracted_data[column].append(row[column_index])

    return extracted_data


# Paths
solo_file_path = "SOLO.xlsx"
bloom_cognitive_file_path = "Bloom_cognitive.xlsx"
bloom_psychomotor_file_path = "Bloom_psychomotor.xlsx"
bloom_affective_file_path = "Bloom_affective.xlsx"
# Verbs
mapped_verbs = {
    "Cognitive": verb_classifier(bloom_cognitive_file_path),
    "Affective": verb_classifier(bloom_affective_file_path),
    "Psychomotor": verb_classifier(bloom_psychomotor_file_path),
    "SOLO": verb_classifier(solo_file_path),
}

verb_list = generate_verb_list(mapped_verbs)
print(verb_list)

# Thresholds to filter the similarity of words to improve accuracy
suggested_sim_threshold = 0.985
sim_threshold = 0.997

# Example usage
csv_file = "Learning outcomes manual mapping - Mappings.csv"
# csv_file = 'Learning outcomes manual mapping - Mappings - Testing.csv'
columns_to_extract = ["LO", "Cognitive", "Affective", "Psychomotor", "SOLO"]

extracted_data = extract_columns(csv_file, columns_to_extract)

sentences = extracted_data["LO"]
final_levels = {
    "Cognitive": extracted_data["Cognitive"],
    "Affective": extracted_data["Affective"],
    "Psychomotor": extracted_data["Psychomotor"],
    "SOLO": extracted_data["SOLO"],
}

CURRENT_MAPPING="Lists_MonashEngineering.docx"
# extract tables from word document
from docx2csv import extract_tables
from docx import Document
tables = extract_tables(CURRENT_MAPPING)
document = Document(CURRENT_MAPPING)

def read_docx_table(document,table_num):
  table = document.tables[table_num-1]
  data = [[cell.text for cell in row.cells] for row in table.rows]
  df = pd.DataFrame(data)
  return df

training_sentences_4 = []

table_num=1
df_po = read_docx_table(document,table_num)
df_po.head(n=12)
for i in range(len(df_po[1])):
  training_sentences_4.append(df_po[1][i].replace("\n",""))

table_num=2
df1 = read_docx_table(document,table_num)
df_clo = df1.copy()
for i in range(len(df_clo[1])):
  training_sentences_4.append(df_clo[1][i].replace("\n",""))

sentences = training_sentences_4
# print(sentences)

nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

stop_words = set(stopwords.words("english"))

# Array of all the PLOs and ULOs (We can couple them together as we're trying to identify Bloom/Solo level here)
lo_sentence_array = []

# training_sentences = training_sentences_1
# training_sentences = training_sentences_2
# training_sentences = training_sentences_3
training_sentences = training_sentences_4

# TODO: train CLO classification with all data instead of just one course.
for sentence in training_sentences:
    sentence = re.sub(r"[^\w\s]", "", sentence)
    # Process the sentence using spaCy
    doc = nlp(sentence)

    # Extract the verbs from the processed sentence
    cleaned_tokens = [token.lemma_ for token in doc if token.lower_ not in stop_words]
    cleaned_tokens = britishise(cleaned_tokens)
    lo_sentence_array.append(cleaned_tokens)

# build the vocabulary and train the model
# IMPORTANT, N0TE THAT sg=1 flag specifies Word2Vec to use the Skip Gram Model as designated by the LSTM paper.
model = Word2Vec(
    sentences=lo_sentence_array,
    vector_size=100,
    window=5,
    min_count=1,
    workers=4,
    sg=1,
    epochs=30,
)

model.build_vocab(corpus_iterable=verb_list, update=True)
model.update_weights()

model_1_wv = model.wv

print("Method 1")
word_vectors = model_1_wv

# print("\nMethod 2")
# word_vectors = model_2_wv
# learning_outcome_mapping(sentences, final_levels, sim_threshold, suggested_sim_threshold)

                          Remembering Understanding     Applying   Analysing  \
Bloom Cognitive Monash 0       define      classify       choose   breakdown   
                       1         draw       compare      compute  categorise   
                       2    duplicate      describe  demonstrate         NaN   
                       3         list       discuss    dramatise    contrast   
                       4        label   distinguish       employ   criticise   
...                               ...           ...          ...         ...   
Bloom Cognitive UIC    31         NaN           NaN          NaN         NaN   
                       32         NaN           NaN        place         NaN   
                       34         NaN           NaN          NaN      survey   
                       36         NaN           NaN     schedule         NaN   
                       37         NaN           NaN    stimulate         NaN   

                          Evaluating   

[nltk_data] Downloading package punkt to /home/arragon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/arragon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/arragon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Method 1


### Executing Learning Outcome Mapping Function

In [95]:
def learning_outcome_mapping(
    sentences, final_levels, SIM_THRESHOLD, SUGGESTED_SIM_THRESHOLD
):
    """
    Input:
        sentences: An array of Learning Outcomes (sentences) in string format.
        final levels: An dictionary of arrays. The dictionary keys are the taxonomies and the arrays contain strings representing the final mapped level of the corresponding learning outcome. If LO is not mapped to that domain leave null value
    """

    mappings = pd.DataFrame(
        columns=[
            "Sentence",
            "Chosen Taxonomy",
            "Mapped Level",
            "Verbs Identified",
            "Suggested Verbs"
        ]
    )

    for i in range(len(sentences)):  # Iterates over the LOs
        sentence = britishise(sentences[i])

        identified_verbs = identify_verbs(sentence)
        similar_verbs = {}

        score_list = {
            "Cognitive": {
                "Remembering": 0,
                "Understanding": 0,
                "Applying": 0,
                "Analysing": 0,
                "Evaluating": 0,
                "Creating": 0,
            },
            "Affective": {
                "Receiving": 0,
                "Responding": 0,
                "Valuing": 0,
                "Organisation": 0,
                "Characterisation": 0,
            },
            "Psychomotor": {
                "Perception": 0,
                "Set": 0,
                "Guided Response": 0,
                "Mechanism": 0,
                "Complex Overt Response": 0,
                "Adaptation": 0,
                "Origination": 0,
            },
            "SOLO": {
                "Prestructural": 0,
                "Unistructural": 0,
                "Multistructural": 0,
                "Relational": 0,
                "Extended Abstract": 0,
            },
        }

        for taxonomy_key, taxonomy_item in mapped_verbs.items():
            similar_verbs[taxonomy_key] = pd.DataFrame(columns=["Level", "Similarity"])

            for identified_verb in identified_verbs:
                for k in range(taxonomy_item.shape[1]):  # Col (Level)
                    for j in range(taxonomy_item.shape[0]):  # Row
                        current_level =taxonomy_item.columns[k]
                        verb = taxonomy_item.iloc[j, k]

                        if verb is None or not verb or pd.isna(verb):
                            continue

                        similarity_score = 0
                        try:  # Currently some of the 'verbs' identified are phrases rather than words and it was throwing errors so this is a temp solution
                            sim_score = word_vectors.similarity(identified_verb, verb)
                            if (
                                sim_score >= SUGGESTED_SIM_THRESHOLD
                                and identified_verb != verb
                            ):
                                similar_verbs[taxonomy_key].at[
                                    verb, "Level"
                                ] = current_level
                                similar_verbs[taxonomy_key].at[
                                    verb, "Similarity"
                                ] = sim_score
                            if sim_score >= SIM_THRESHOLD:
                                similarity_score += sim_score
                        except:
                            pass
                        score_list[taxonomy_key][
                            current_level
                        ] += similarity_score

        # Determine Taxonomy and Identify level based on similarity
        max_score = {
            "Cognitive": {"Level": None, "Score": 0},
            "Affective": {"Level": None, "Score": 0},
            "Psychomotor": {"Level": None, "Score": 0},
            "SOLO": {"Level": None, "Score": 0},
        }
        for t_key, t_item in score_list.items():
            x = mappings.shape[0]

            for l in t_item:
                if max_score[t_key]["Score"] < score_list[t_key][l]:
                    max_score[t_key] = {"Level": l, "Score": score_list[t_key][l]}

            if max_score[t_key]["Level"] != None:
                # Generate output for sentence
                mappings.at[x, "Index"] = i
                mappings.at[x, "Sentence"] = sentence
                mappings.at[x, "Chosen Taxonomy"] = t_key
                mappings.at[x, "Mapped Level"] = max_score[t_key]["Level"]

                # Generate identified verb and level tuples for sentence data
                s_d_identified_verbs = []
                for verb in identified_verbs:
                    level = "Verb not mapped"
                    try:
                        level = verb_list.at[(t_key, verb), "Level"]
                    except:
                        pass
                    finally:
                        s_d_identified_verbs.append((verb, level))

                mappings.at[x, "Verbs Identified"] = s_d_identified_verbs

                # Generate suggested verbs
                sim_verbs = (
                    similar_verbs[t_key]
                    .sort_values(by=["Similarity"], ascending=False)
                    .head(5)
                )
                suggested_verbs = []
                if sim_verbs.shape[0] > 0:
                    suggested_verbs = [
                        (verb, sim_verbs.at[verb, "Level"]) for verb in sim_verbs.index
                    ]
                mappings.at[x, "Suggested Verbs"] = suggested_verbs

    return mappings

# TODO: Classify the verbs in each of the learning outcomes
ulo_sentences = p1[1].to_list()
plo_sentences = df[1].to_list()

# Thresholds to filter the similarity of words to improve accuracy
suggested_sim_threshold = 0.985
sim_threshold = 0.997

# Example usage
csv_file = "Learning outcomes manual mapping - Mappings.csv"
# csv_file = 'Learning outcomes manual mapping - Mappings - Testing.csv'
columns_to_extract = ["LO", "Cognitive", "Affective", "Psychomotor", "SOLO"]

extracted_data = extract_columns(csv_file, columns_to_extract)

final_levels = {
    "Cognitive": extracted_data["Cognitive"],
    "Affective": extracted_data["Affective"],
    "Psychomotor": extracted_data["Psychomotor"],
    "SOLO": extracted_data["SOLO"],
}

ulo_classifications = learning_outcome_mapping(ulo_sentences, final_levels, sim_threshold, suggested_sim_threshold)
plo_classifications = learning_outcome_mapping(plo_sentences, final_levels, sim_threshold, suggested_sim_threshold)

# Iteration 1: Assume both CLOs and POs are classifiable into Cognitive, Affective, Psychomotor Levels.
# If levels match, boost the CLO to PO coefficient by 0.1
# If levels different, don't boost

In [96]:

# print(ulo_classifications["Mapped Level"][0])
# print(plo_classifications["Mapped Level"][0])
print(ulo_classifications)

                                              Sentence Chosen Taxonomy  \
0    describe the underlying theoretical basis of t...       Cognitive   
1    describe the underlying theoretical basis of t...       Affective   
2    describe the underlying theoretical basis of t...     Psychomotor   
3    describe the underlying theoretical basis of t...            SOLO   
4    design a relational database model based on th...       Cognitive   
..                                                 ...             ...   
333  explain the contract laws and apply ethical pr...     Psychomotor   
334  explain the contract laws and apply ethical pr...            SOLO   
335  communicate effectively on complex civil engin...       Cognitive   
336  communicate effectively on complex civil engin...       Affective   
337  communicate effectively on complex civil engin...            SOLO   

          Mapped Level                                   Verbs Identified  \
0        Understanding  [(describe

### ADJUSTING THRESHOLDS USING THE LEVELS FOUND FOR LO Mapper & PO Mapper.

In [97]:
# for classification in ulo_classifications:
#     print(classification)

# BASE ALGORITHM ACCURACY (w/ no learning outcome mapper), ADJUSTMENT_THRESHOLD=0:
# 0.8544061302681993

# ALGORITHM ACCURACY (w/ new learning outcome mapper), ADJUSTMENT_THRESHOLD=0.1:
# 0.8927203065134102
# ALGORITHM ACCURACY (w/ new learning outcome mapper), ADJUSTMENT_THRESHOLD=0.15:
# 0.8936781609195404
# ALGORITHM ACCURACY (w/ new learning outcome mapper), ADJUSTMENT_THRESHOLD=0.2:
# 0.8946360153256706
# ALGORITHM ACCURACY (w/ new learning outcome mapper), ADJUSTMENT_THRESHOLD=0.25:
# 0.8946360153256707
# ALGORITHM ACCURACY (w/ new learning outcome mapper), ADJUSTMENT_THRESHOLD=0.4:
# 0.8764367816091956
# ALGORITHM ACCURACY (w/ new learning outcome mapper), ADJUSTMENT_THRESHOLD=0.5:
# 0.8678160919540232
# ALGORITHM ACCURACY (w/ new learning outcome mapper), ADJUSTMENT_THRESHOLD=1:
# 0.8678160919540232

ADJUSTMENT_THRESHOLD = 0.25

# Goes through each cell in the LO->PO Mapping table and adjusts the value using a static ADJUSTMENT_THRESHOLD Value.
for x in range(len(plo_classifications['Mapped Level'])):
    for y in range(len(ulo_classifications['Mapped Level'])):
        i = int(plo_classifications.at[x, "Index"])
        j = int(ulo_classifications.at[y, "Index"])
        if ulo_classifications['Mapped Level'][y] == plo_classifications['Mapped Level'][x]:
            if dd['PO'+str(i+1)][j] + ADJUSTMENT_THRESHOLD <= 1:
                dd['PO'+str(i+1)][j] += ADJUSTMENT_THRESHOLD # Add Offset
            else:
                dd['PO'+str(i+1)][j] = 1
        else:
            if dd['PO'+str(i+1)][j] - ADJUSTMENT_THRESHOLD >= 0:
                dd['PO'+str(i+1)][j] -= ADJUSTMENT_THRESHOLD # Add Offset
            else:
                dd['PO'+str(i+1)][j] = 0

/tmp/ipykernel_11207/2445793229.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dd['PO'+str(i+1)][j] = 0
/tmp/ipykernel_11207/2445793229.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dd['PO'+str(i+1)][j] += ADJUSTMENT_THRESHOLD # Add Offset
/tmp/ipykernel_11207/2445793229.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dd['PO'+str(i+1)][j] -= ADJUSTMENT_THRESHOLD # Add Offset
/tmp/ipykernel_11207/2445793229.py:33: Sett

In [98]:
# store the matrix into csv file
dd.to_csv('pseudocodematrix.csv', index=False)

## Setting threshold value (taking min and max of each column and divided by 2)
## threshold value = (min +max)/2

In [99]:

# TODO: Alter the threshold based on matching hierarchy type & bloom verb instead of simply using (column_max+column_min)/2

# Setting threshold value
# Taking min max average of each column and set that as a threshold value

# This will change the coefficients into 0 or 1 mappings in the dd dataframe
for x in range(len(df)):
    tes = dd['PO'+str(x+1)].values.min()
    tes1 = dd['PO'+str(x+1)].values.max()
    tt1 = (tes+tes1)/2

    if tt1 == 0:
      dd['PO'+str(x+1)] = dd['PO'+str(x+1)]
    else:
      dd['PO'+str(x+1)] = dd['PO'+str(x+1)].apply(lambda x: 1 if x >= tt1 else 0)
# dd

/tmp/ipykernel_11207/2949801979.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dd['PO'+str(x+1)] = dd['PO'+str(x+1)].apply(lambda x: 1 if x >= tt1 else 0)
/tmp/ipykernel_11207/2949801979.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dd['PO'+str(x+1)] = dd['PO'+str(x+1)].apply(lambda x: 1 if x >= tt1 else 0)
/tmp/ipykernel_11207/2949801979.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [100]:
# dd

dd.to_csv('PLO-CLOmapping.csv', index=False)

In [101]:
dd.head()

,PO1,PO2,PO3,PO4,PO5,PO6,PO7,PO8,PO9,PO10,PO11,PO12
0,0,0,0,0,1,1,0,0,0,0,1,1
1,0,0,1,0,0,0,0,0,1,0,0,0
2,1,1,0,1,0,0,1,1,0,1,0,0
3,1,1,0,1,0,0,1,1,0,1,0,0
4,0,0,0,0,1,1,0,0,0,0,1,1


In [102]:
# human generated output
d= pd.read_csv(ORIGINAL_MAPPING)
d.head()

,PO1,PO2,PO3,PO4,PO5,PO6,PO7,PO8,PO9,PO10,PO11,PO12
0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0


In [103]:
df3 = d.copy()

In [104]:
dd

,PO1,PO2,PO3,PO4,PO5,PO6,PO7,PO8,PO9,PO10,PO11,PO12
0,0,0,0,0,1,1,0,0,0,0,1,1
1,0,0,1,0,0,0,0,0,1,0,0,0
2,1,1,0,1,0,0,1,1,0,1,0,0
3,1,1,0,1,0,0,1,1,0,1,0,0
4,0,0,0,0,1,1,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
82,1,1,0,1,0,0,1,1,0,1,0,0
83,0,0,0,0,1,1,0,0,0,0,1,1
84,1,1,0,1,0,0,1,1,0,1,0,0
85,1,1,0,1,0,0,1,1,0,1,0,0


In [105]:
df3

,PO1,PO2,PO3,PO4,PO5,PO6,PO7,PO8,PO9,PO10,PO11,PO12
0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
82,0,0,0,0,0,0,0,0,0,0,0,1
83,0,0,0,0,0,1,0,0,0,0,0,0
84,0,0,0,0,0,0,1,0,0,0,0,0
85,0,0,0,0,0,0,0,1,0,0,0,0


In [106]:
for x in range(len(df)):
  df3['PO'+str(x+1)] = np.where(dd['PO'+str(x+1)] == df3['PO'+str(x+1)], 'True', 'False')

In [107]:
df3.head()

,PO1,PO2,PO3,PO4,PO5,PO6,PO7,PO8,PO9,PO10,PO11,PO12
0,False,True,True,True,False,False,True,True,True,True,False,False
1,True,False,False,True,True,True,True,True,False,True,True,True
2,False,False,False,False,True,True,False,False,True,False,True,True
3,True,False,True,False,True,True,False,False,True,False,True,True
4,True,True,False,True,False,False,True,True,True,True,False,False


In [108]:
for x in range(len(df)):
  df3['PO'+str(x+1)] = df3['PO'+str(x+1)].replace('True', 1)
  df3['PO'+str(x+1)] = df3['PO'+str(x+1)].replace('False', 0)

In [109]:
df3.head()

,PO1,PO2,PO3,PO4,PO5,PO6,PO7,PO8,PO9,PO10,PO11,PO12
0,0,1,1,1,0,0,1,1,1,1,0,0
1,1,0,0,1,1,1,1,1,0,1,1,1
2,0,0,0,0,1,1,0,0,1,0,1,1
3,1,0,1,0,1,1,0,0,1,0,1,1
4,1,1,0,1,0,0,1,1,1,1,0,0


In [110]:
# calculating accuracy of the table
df3['acc'] = df3.mean(axis=1)
df3.head(n=100)

,PO1,PO2,PO3,PO4,PO5,PO6,PO7,PO8,PO9,PO10,PO11,PO12,acc
0,0,1,1,1,0,0,1,1,1,1,0,0,0.583333
1,1,0,0,1,1,1,1,1,0,1,1,1,0.750000
2,0,0,0,0,1,1,0,0,1,0,1,1,0.416667
3,1,0,1,0,1,1,0,0,1,0,1,1,0.583333
4,1,1,0,1,0,0,1,1,1,1,0,0,0.583333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,0,0,1,0,1,1,0,0,1,0,1,0,0.416667
83,1,1,1,1,0,1,1,1,1,1,0,0,0.750000
84,0,0,1,0,1,1,1,0,1,0,1,1,0.583333
85,0,0,1,0,1,1,0,1,1,0,1,1,0.583333


In [111]:
df4 = pd.concat([df1[0], df3], axis=1)
df4.head(n=100)

,0,PO1,PO2,PO3,PO4,PO5,PO6,PO7,PO8,PO9,PO10,PO11,PO12,acc
0,LO1-MA,0,1,1,1,0,0,1,1,1,1,0,0,0.583333
1,LO2-MA,1,0,0,1,1,1,1,1,0,1,1,1,0.750000
2,LO3-MA,0,0,0,0,1,1,0,0,1,0,1,1,0.416667
3,LO4-MA,1,0,1,0,1,1,0,0,1,0,1,1,0.583333
4,LO5-MA,1,1,0,1,0,0,1,1,1,1,0,0,0.583333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,LO2-MA,0,0,1,0,1,1,0,0,1,0,1,0,0.416667
83,LO3-MA,1,1,1,1,0,1,1,1,1,1,0,0,0.750000
84,LO4-MA,0,0,1,0,1,1,1,0,1,0,1,1,0.583333
85,LO5-MA,0,0,1,0,1,1,0,1,1,0,1,1,0.583333


In [112]:
df4.set_index(0, inplace=True)
df4.head(n=100)

df4.to_csv('WasMappingSuccessful.csv')

In [113]:
df4['acc'].mean()

0.5833333333333334